# Chess Engine with Pytorch

### Imports

In [1]:
import os 
import numpy
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from chess import pgn
from tqdm import tqdm

### Data Preprocessing


#### Load Data

In [2]:
def load_pgn(file_path):
    games = []
    with open(file_path, "r") as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)	
    return games

files = [file for file in os.listdir("../Database") if file.endswith(".pgn")]

LIMIT_OF_FILES = min(len(files), 20)

games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"../Database/{file}"))
    if i == LIMIT_OF_FILES:
        break
    i += 1


 17%|█▋        | 19/110 [00:21<01:41,  1.11s/it]


In [3]:
print(f"Games Parsed: {len(games)}")

Games Parsed: 5667


# Convert data into tensors

In [4]:
from auxiliary_functs import create_input_for_nn, encode_moves

In [5]:
X, y = create_input_for_nn(games)
print(f"Number of samples: {len(y)}")

Number of samples: 479333


In [6]:
# X = X[0:2500000]
# y = y[0:2500000]

In [7]:
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [8]:
X = torch.tensor(X,dtype=torch.float32)
y = torch.tensor(y,dtype=torch.long)


# Preliminary actions

In [9]:
from dataset import ChessDataset
from model import ChessModel

In [10]:
# Create Dataset and DataLoader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Load the model state
if os.path.exists("../model/chess_model_pytorch.pth"):
    model.load_state_dict(torch.load("../model/chess_model_pytorch.pth"))
    print("Loaded model from disk.")

# Load the optimizer state
if os.path.exists("../model/chess_optimizer_pytorch.pth"):
    optimizer.load_state_dict(torch.load("../model/chess_optimizer_pytorch.pth"))
    print("Loaded optimizer state from disk.")

Using device: cuda


# Training with games

In [11]:

num_epochs = 50

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device) # move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs) # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        running_loss += loss.item()
    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time % 60)

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss / len(dataloader):.4f} - Time: {minutes}m {seconds}s")

100%|██████████| 7490/7490 [01:24<00:00, 88.28it/s] 


Epoch 1/50 - Loss: 4.8225 - Time: 1m 24s


100%|██████████| 7490/7490 [01:26<00:00, 86.39it/s] 


Epoch 2/50 - Loss: 3.6173 - Time: 1m 26s


100%|██████████| 7490/7490 [01:25<00:00, 87.10it/s] 


...

Epoch 48/50 - Loss: 0.4341 - Time: 1m 3s


100%|██████████| 7490/7490 [01:03<00:00, 117.59it/s]


Epoch 49/50 - Loss: 0.4229 - Time: 1m 3s


100%|██████████| 7490/7490 [01:03<00:00, 117.47it/s]


Epoch 50/50 - Loss: 0.4115 - Time: 1m 3s


In [12]:
torch.save(model.state_dict(), "../model/chess_model_pytorch.pth")
torch.save(optimizer.state_dict(), "../model/chess_optimizer_pytorch.pth")

In [13]:
import pickle

with open("../model/move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)